In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 23.9 MB/s eta 0:00:01   |▊                               | 552 kB 2.8 MB/s eta 0:00:09
     |████████████████████████████████| 981 kB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 263 kB 51.9 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 428 kB 43.1 MB/s eta 0:00:01


  Created wheel for ktrain: filename=ktrain-0.25.0-py3-none-any.whl size=25274500 sha256=f981d2cf776ae1777e26ee435aea21cdf22d8ee5644a8da7814bc2bebbf0ba96
  Stored in directory: /root/.cache/pip/wheels/ef/4b/a8/7ef8b32fcba22ad93b43eefd341ba451f80d9ef1c24e5c6061
  Created wheel for keras-bert: filename=keras_bert-0.86.0-py3-none-any.whl size=34147 sha256=9c7d3a640d4a6f785d34277e85da22f8af887a3ce258bcbcf8f9e64b26d4ce1c
  Stored in directory: /root/.cache/pip/wheels/fc/c1/0a/eb9187261b3f192ac314aefb54fe66f50540c3edb906599633
  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993191 sha256=fc68863323803f55088786aac020c6ca055702204932f3a7b11a124a1146db1a
  Stored in directory: /root/.cache/pip/wheels/59/f6/9d/85068904dba861c0b9af74e286265a08da438748ee5ae56067
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9917 sha256=f4b857d0a7e2ba1e5908cdffa15837b21829b7884635fa714b55ed563960b805
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
tf.__version__

'2.3.1'

In [5]:
import os
print(os.listdir("../input/usi-nlp-practicum-2"))
import warnings
warnings.filterwarnings('ignore')

['glove.6B.50d.txt', 'sample_submission.csv', 'imdb_train.csv', 'winequality-red.csv', 'imdb_test_data.csv', 'winequality-white.csv']


In [6]:
#importing the training data
data_train = pd.read_csv('../input/usi-nlp-practicum-2/imdb_train.csv')
print(data_train.shape)
print(data_train.head(10))
#importing the training data
data_test=pd.read_csv('../input/usi-nlp-practicum-2/imdb_test_data.csv')
print(data_test.shape)
print(data_test.head(10))

(40000, 2)
                                              review sentiment
0  We had STARZ free weekend and I switched on th...  negative
1  I'll admit that this isn't a great film. It pr...  negative
2  I finally found a version of Persuasion that I...  positive
3  The BBC surpassed themselves with the boundari...  positive
4  Much praise has been lavished upon Farscape, b...  negative
5  Of course the plot, script, and, especially ca...  positive
6  This is one of those road movies that would li...  negative
7  What an uninteresting hodge-podge. It could ha...  negative
8  Only a handful of the segments are engaging he...  negative
9  THE GREEN BUTCHERS (Anders Thomas Jensen - Den...  positive
(10000, 2)
   id                                             review
0   1  Not only is this movie a great film for basic ...
1   2  Waitress: Honey, here's them eggs you ordered....
2   3  Many mystery stories follow the standard whodu...
3   4  A space ship cruising through the galaxy encou...


In [7]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)


Size of train dataset:  (40000, 2)
Size of test dataset:  (10000, 2)


In [8]:
#sentiment count
data_train['sentiment'].value_counts()

positive    20048
negative    19952
Name: sentiment, dtype: int64

In [9]:

(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'review',
                                                                   label_columns = 'sentiment',
                                                                   val_df = data_test,
                                                                   
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [10]:
X_test

[array([[  101,  2025,  2069, ...,     0,     0,     0],
        [  101, 13877,  1024, ...,  2003,  2055,   102],
        [  101,  2116,  6547, ...,     0,     0,     0],
        ...,
        [  101,  2108,  1037, ...,     0,     0,     0],
        [  101,  7929,  2034, ...,     0,     0,     0],
        [  101,  7910, 23644, ...,  2143,  1010,   102]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [11]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500
done.


In [12]:
#here we have taken batch size as 6 as from the documentation it is recommend to use this with maxlen as 500

learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   
                   batch_size = 6)

In [13]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('bert')



begin training using onecycle policy with max lr of 2e-05...
6667/6667 [==============================] - 5397s 810ms/step - loss: 0.2228 - accuracy: 0.9103


In [16]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [17]:
predictor.predict(data)

['negative', 'negative', 'positive']

In [19]:
dflist=data_test['review'].to_list()


In [ ]:
data_test['prediction']=predictor.predict(dflist)
predictor.get_classes()

In [ ]:
data_test.head()

In [ ]:
import os
os.chdir(r'/kaggle/working')
predictor.save('/kaggle/working/bert')
data_test.to_excel('/kaggle/working/Prediction.xlsx')
!zip -r file.zip '/kaggle/working/'
print(os.listdir())
from IPython.display import FileLink
FileLink('file.zip')

test_data.to_excel('Output.xlsx')